In [ ]:
import os
import json

from paquo.projects import QuPathProject
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import numpy as np
import PIL
from PIL import Image
from openslide import OpenSlide
import shapely
from shapely.geometry import Polygon

# open the QuPath project and print classes

In [ ]:
qp = QuPathProject('./Qupath_data/project.qpproj', mode = 'r+')
qp_class_dic = {}
for i, ann_cls in enumerate(qp.path_classes):
    qp_class_dic[ann_cls.id] = i
qp_class_dic

# Set Prams, open the slide and get Tile in Slide

In [ ]:
# tile parameter
# pixel/ microm = 4.3
num_classes = len(qp_class_dic)
location_x, location_y, size = (50000, 8000, 4096) 

In [ ]:
slide = qp.images[0]  # get the first image
with OpenSlide(os.path.join("Qupath_data", "Slides zum Training", slide.image_name)) as slide_data:
# slide_data = openslide.OpenSlide(os.path.join("Qupath_data", "Slides zum Training", slide.image_name))
    img_data = slide_data.read_region((location_x, location_y), 0, (size, size))
plt.imshow(np.array(img_data))

# Detect intersections between img tile and annotations

In [ ]:
slide = qp.images[0]
hier_data = slide.hierarchy.to_geojson()
geo_list_tile = []
tile_intersections = []
polygon_tile = Polygon(([location_x, location_y], [location_x + size, location_y], [location_x + size, location_y + size], [location_x, location_y + size]))
for geojson in hier_data:
    try:
        annot_class = geojson["properties"]["classification"]["name"]
    except KeyError:
        continue
    poly_data = geojson['geometry']['coordinates']
    try:    
        polygon_annot = Polygon(poly_data[0], poly_data[1:]) if len(poly_data) > 1 else Polygon(poly_data[0])
    except (ValueError, AssertionError) as e:
        poly_data_squeezed = []
        for i in range(len(poly_data)):
            poly_data_squeezed.append(np.array(poly_data[i]).squeeze())
        try:
            polygon_annot = Polygon(poly_data_squeezed[0], poly_data_squeezed[1:]) if len(poly_data_squeezed) > 1 else Polygon(poly_data_squeezed[0])
        except ValueError:
            poly_data_squeezed[0] = poly_data_squeezed[0][0]
    
    intersection = polygon_annot.intersection(polygon_tile)
    if not intersection.is_empty:
        tile_intersections.append((annot_class, intersection))
print(len(tile_intersections))

# read in intersection(s) as new annotations in the QuPath Project

In [ ]:
# for intersection in tile_intersections:
#     slide.hierarchy.add_annotation(intersection, path_class= qp.path_classes[12])
# qp.save()

# Visualize annotations in tile (size,size)

In [ ]:
## translate the annot for tile
# tile_annot = []
img_mask = np.zeros((size,size))
for i, intersection_data in enumerate(tile_intersections):
    _, intersection = intersection_data
    trans_inter = shapely.affinity.translate(intersection, location_x * -1, location_y * -1)
    # tile_annot.append(translated_intersection)

    ## vis annotations in tile
    int_coords = lambda coords: np.array(coords).round().astype(np.int32)
    exteriors = [int_coords(trans_inter.exterior.coords)]
    interiors = [int_coords(poly) for poly in trans_inter.interiors]
    
    cv2.fillPoly(img_mask, exteriors, i + 1)
    cv2.fillPoly(img_mask, interiors, 0)
plt.imshow(img_mask)
plt.show()

# Extract tile and tile anotations to numpy arrays
## shape picture: (size, size)
## shape annotations: (num_class, size, size) 

In [ ]:
label_img = np.zeros((num_classes,size,size))
for inter_class, intersection in tile_intersections:
    class_num = qp_class_dic[inter_class]
    label_mask = label_img[class_num]

    trans_inter = shapely.affinity.translate(intersection, location_x * -1, location_y * -1)

    int_coords = lambda coords: np.array(coords).round().astype(np.int32)
    exteriors = [int_coords(trans_inter.exterior.coords)]
    interiors = [int_coords(poly) for poly in trans_inter.interiors]
    
    cv2.fillPoly(label_mask, exteriors, 1)
    cv2.fillPoly(label_mask, interiors, 0)

    label_img[class_num] = label_mask